# Transfer Learning: Feature Extraction, Function API

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

 98% 325M/331M [00:03<00:00, 142MB/s]
100% 331M/331M [00:03<00:00, 108MB/s]


In [ ]:
import zipfile
zip = zipfile.ZipFile('/content/utkface-new.zip')
zip.extractall('/content')
zip.close()

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'
folder_path

'/content/utkface_aligned_cropped/UTKFace'

# predict - age, gender, and ethnicity
# age - regression problem
# gender - binary classification
# ethnicity - multiclass classification problem

In [ ]:
age = []
gender = []
ethnicity = [] # you have to do it
img_path = []

for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)


In [ ]:
len(age)

23708

In [ ]:
len(gender)

23708

In [ ]:
df = pd.DataFrame({'age': age, 'gender':gender, 'img_path': img_path})

In [ ]:
df

,age,gender,img_path
0,26,1,26_1_3_20170104232413655.jpg.chip.jpg
1,2,0,2_0_4_20161221195155711.jpg.chip.jpg
2,11,1,11_1_2_20170104005111615.jpg.chip.jpg
3,8,1,8_1_0_20170109204933562.jpg.chip.jpg
4,28,0,28_0_4_20170109140345949.jpg.chip.jpg
...,...,...,...
23703,26,1,26_1_3_20170119193136450.jpg.chip.jpg
23704,50,0,50_0_0_20170117160624326.jpg.chip.jpg
23705,32,0,32_0_2_20170116184944206.jpg.chip.jpg
23706,24,1,24_1_3_20170119171104872.jpg.chip.jpg


In [ ]:
df.shape

(23708, 3)

### split the data into training and validation

In [ ]:
train_df = df.sample(frac=1, random_state=0).iloc[:20000]
test_df = df.sample(frac=1, random_state=0).iloc[20000:]

In [ ]:
print(train_df.shape, test_df.shape)

(20000, 3) (3708, 3)


In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=25,
                                   width_shift_range=0.2,height_shift_range=0.2,
                                   zoom_range=0.2,horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255,rotation_range=25,
                                   width_shift_range=0.2,height_shift_range=0.2,
                                   zoom_range=0.2,horizontal_flip=True)

In [ ]:
# apply data augmentaion with original dataset
train_generator = train_datagen.flow_from_dataframe(train_df, directory=folder_path,
                                                    x_col = 'img_path', y_col=['age','gender'],
                                                    target_size = (256, 256),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df, directory=folder_path,
                                                    x_col = 'img_path', y_col=['age','gender'],
                                                    target_size = (256, 256),
                                                    class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


# ResNet Model - Transfer Learning

In [ ]:
from keras.layers import *
from keras.models import Model
from keras.applications.resnet50 import ResNet50

In [ ]:
resnet = ResNet50(include_top=False, input_shape=(256, 256, 3))

In [ ]:
resnet = ResNet50(include_top=False, input_shape=(256, 256, 3))
resnet.trainable  = False

output = resnet.layers[-1].output
flatten = Flatten()(output)

dense1 = Dense(128, activation='relu')(flatten)
dense2 = Dense(128, activation='relu')(flatten)

dense3 = Dense(64, activation='relu')(dense1)
dense4 = Dense(64, activation='relu')(dense2)

output1 = Dense(1, activation='linear', name='age')(dense3)
output2 = Dense(1, activation='sigmoid', name='gender')(dense4)

model = Model(inputs=resnet.input, outputs=[output1, output2])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
model.compile(optimizer='adam', loss={'age':'mae', 'gender':'binary_crossentropy'},
              metrics={'age':'mae', 'gender':'accuracy'},
              loss_weights={'age':10, 'gender':90})

In [ ]:
history = model.fit(train_generator, batch_size=32, epochs=5,
                    validation_data=test_generator)

Epoch 1/5
566/625 [==========================>...] - ETA: 32s - loss: 227.8618 - age_loss: 15.3351 - gender_loss: 0.8279 - age_mae: 15.3351 - gender_accuracy: 0.5213